# Analyzing the Metadata of Publications in the Open Research Knowledge Graph 
This Jupyter notebook contains different analyses of scholarly knowledge from scientific publications on the topic of *Tailored Forming Processes for Manufacturing Components with Bearing Raceways Using Different Material Combinations* from the research field *Mechanical Process Engineering*. These analyses are based on domain-specific questions on the described scholarly knowledge raised by domain experts in this research area. While the individual questions are presented below for overview, their details are explained in the respective section of this Jupyter notebook.

## Loading data from ORKG SPAQRL endpoint
First, the metadata of the publications in the *Open Research Knowledge Graph*  ([ORKG](https://www.orkg.org/orkg/)) is loaded.

In [86]:
%matplotlib widget
from SPARQLWrapper import SPARQLWrapper, CSV
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

ENDPOINT_URL = "https://www.orkg.org/orkg/triplestore"

PREFIXES =  """
            PREFIX orkgr: <http://orkg.org/orkg/resource/>
            PREFIX orkgc: <http://orkg.org/orkg/class/>
            PREFIX orkgp: <http://orkg.org/orkg/predicate/>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            """

query = """
        SELECT ?paper, ?field_label, ?DOI, ?title, ?name, ?id, ?month_number, ?year_number, ?paper_url, ?venue_label
WHERE {
  ?paper a orkgc:Paper.
  OPTIONAL{?paper rdfs:label ?title.}
  OPTIONAL{?paper orkgp:P26 ?DOI.}
  OPTIONAL{?paper orkgp:P30 ?field.
           ?field rdfs:label ?field_label.}
  OPTIONAL{?paper orkgp:P27 ?author.
           BIND(IF(isLiteral(?author), ?author, "") AS ?name1)
           OPTIONAL{?author rdfs:label ?author_label;
                            orkgp:HAS_ORCID ?id.}
           BIND(IF(BOUND(?author_label),?author_label, "") AS ?name2)
           BIND(IF(?name1 = "", ?name2, ?name1) AS ?name)
          }
  
  OPTIONAL{?paper orkgp:P28 ?month.
           BIND(IF(isLiteral(?month), ?month, "") AS ?month1)
           OPTIONAL{?month rdfs:label ?month_label.}
           BIND(IF(BOUND(?month_label),?month_label, "") AS ?month2)
           BIND(IF(?month1 = "", ?month2, ?month1) AS ?month_number)
          }
  
  OPTIONAL{?paper orkgp:P29 ?year.
           BIND(IF(isLiteral(?year), ?year, "") AS ?year1)
           OPTIONAL{?year rdfs:label ?year_label.}
           BIND(IF(BOUND(?year_label),?year_label, "") AS ?year2)
           BIND(IF(?year1 = "", ?year2, ?year1) AS ?year_number)
          }
  
  OPTIONAL{?paper orkgp:url ?paper_url.}
  OPTIONAL{?paper orkgp:HAS_VENUE ?venue.
           ?venue rdfs:label ?venue_label.}
} ORDER BY ?paper
        """

user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])

sparql = SPARQLWrapper(ENDPOINT_URL, agent=user_agent)
sparql.setQuery(PREFIXES+query)
sparql.setReturnFormat(CSV)

try:
        results = sparql.queryAndConvert()
except Exception as e:
        print(e)

now = datetime.now()
with open('query_result_' + now.strftime('%Y-%m-%d') + '.csv', 'wb') as file:
        file.write(results)

#df = pd.read_csv('CLEAN-query-a_paper_and_all_metadata_optional_result_1-41364-2022.05.04.csv')
md_df = pd.read_csv('query_result_' + now.strftime('%Y-%m-%d') + '.csv', encoding='utf-8', encoding_errors='ignore')

print(md_df['paper'].nunique())

9880


## 1. Question: How often are which qualities of the manufactured components examined in which tailored forming process steps?
Based on this question, we want to know which qualities of the manufactured components are typically reported in which steps of the tailored forming process. On the one hand, this information shows which qualities are prominently reported in publications on the tailored forming process. On the other hand, the answer to this question also highlights to which steps generally and for specific qualities little research has been reported so far.

In [100]:

field_df = md_df[['paper','field_label']].drop_duplicates()
print(field_df.isnull().sum())

field_df.fillna('No research field')
field_df[['field_label']].value_counts().head(15)

paper            0
field_label    108
dtype: int64


field_label                          
Science                                  3216
Bioinformatics                           1188
Ecology and Evolutionary Biology          968
Information Science                       324
Artificial Intelligence                   304
Toxicology                                295
Computer Sciences                         238
Urban Studies and Planning                203
Medicinal Chemistry and Pharmaceutics     187
Natural Language Processing               160
Oceanography                              126
Engineering                               108
Virology                                  106
Plasma and Beam Physics                    98
Databases/Information Systems              96
dtype: int64